In [3]:
from dotenv import load_dotenv

load_dotenv()



True

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini")

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

In [7]:
from langchain_core.messages import HumanMessage

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


Failed to multipart ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ProxyError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))"))
Content-Length: 35178
API Key: lsv2_********************************************5btrace=47e66035-52a2-45e1-977b-572f7014ff9c,id=47e66035-52a2-45e1-977b-572f7014ff9c; trace=47e66035-52a2-45e1-977b-572f7014ff9c,id=58e8f496-00ef-4bee-9ed4-943c3490f020; trace=47e66035-52a2-45e1-977b-572f7014ff9c,id=3ee58121-3428-486d-a276-615d66f09b63; trace=47e66035-52a2-45e1-977b-572f7014ff9c,id=6c44affc-36e7-4c56-bd04-4bc2917427a7; trace=47e66035-52a2-45e1-977b-572f7014ff9c,id=34a7d261-fb5b-43b4-a4fd-ecf37f3fa805; trace=47e66035-5

In [8]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today?


## 添加PromptTemplates

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [10]:

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Jim! What brings ye to these treacherous waters today? Be ye seekin' treasure or just a bit o' good ol' pirate chatter? Arrr!


In [12]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be callin' yerself Jim, matey! A fine name fer a landlubber or a scallywag alike! What else can I assist ye with on this fine day? Arrr!


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


## Managing Conversation History

In [18]:
from langchain_core.messages import SystemMessage, trim_messages, AIMessage, HumanMessage


trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [19]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [20]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t know your name. You haven’t told me yet!


In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

It seems you haven't asked a specific math problem yet. If you have a particular math question or problem you'd like assistance with, please provide the details, and I'll do my best to help you!


In [27]:
output

{'messages': [SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}, id='1c9c5e71-68af-4143-89c0-1871001a33a0'),
  HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='9ea35c07-58d1-466e-b68b-54e793a93c7b'),
  AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='09778efa-4303-4a1e-a973-45f80ddbfb02'),
  HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}, id='53e9db1b-070a-4330-bc0b-9f7aa17b99ee'),
  AIMessage(content='nice', additional_kwargs={}, response_metadata={}, id='d89302de-afe4-406f-9edb-439f5e993375'),
  HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}, id='bad41ccf-ec63-4955-9d13-c0d6d504b267'),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}, id='0a8e1faa-1b1d-407a-991a-f39820faf553'),
  HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}, id='ceebd4cc-a5fa-40c0-bc2d-baaac49e0

## Stream

In [22]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| don|’t| scientists| trust| atoms|?

|Because| they| make| up| everything|!||

In [29]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')

for message in state["messages"]:
    message.pretty_print()


Language: English
================================ System Message ================================

you're a good assistant
================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

hi!
================================ Human Message =================================

I like vanilla ice cream
================================== Ai Message ==================================

nice
================================ Human Message =================================

whats 2 + 2
================================== Ai Message ==================================

4
================================ Human Message =================================

thanks
================================== Ai Message ==================================

no problem!
================================ Human Message =================================

having fun?
================================== Ai Message =

In [30]:
from langchain_core.messages import HumanMessage

_ = app.update_state(config, {"messages": [HumanMessage(content="hi")]})


In [31]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')

for message in state["messages"]:
    message.pretty_print()

Language: English
================================ System Message ================================

you're a good assistant
================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

hi!
================================ Human Message =================================

I like vanilla ice cream
================================== Ai Message ==================================

nice
================================ Human Message =================================

whats 2 + 2
================================== Ai Message ==================================

4
================================ Human Message =================================

thanks
================================== Ai Message ==================================

no problem!
================================ Human Message =================================

having fun?
================================== Ai Message =